## 1.  Построение пайплайна для обучения модели. Реализована функция, которая:

1. Принимает на вход тип модели машинного обучения в виде строки и предполагает возможность
выбора одного из трех вариантов (должно быть предусмотрено не менее трех возможных
моделей для обучения)
2. Принимает в качестве аргумента **kwargs словарь с гиперпараметрами для выбранного
алгоритма
3. Выполняет обучение алгоритма машинного обучения на обучающем наборе данных
4. При необходимости промежуточной оценки точности использует тестовый набор данных
5. Проводит оценку точности на валидационной выборке и
6. В качестве результата своей работы возвращает список с:
a. Обученной моделью
b. Словарем с метриками, рассчитанными по валидационному табору данных
(accuracy, roc_auc_score, f1_score (macro))
c. Временем, затраченным на обучение в секундах в расчете на одну эпоху (если
применимо)
d. Временем, затраченным на обучение в минутах
7. А также чтобы каждый запуск сохранял новый файл с временем запуска пайплайна в названии
файла и содержащим информацию, возвращаемую пайплайном

In [26]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from datetime import datetime

In [27]:
#Принимает на вход тип модели машинного обучения в виде строки и **kwargs 
def train_model(model_type: str, data_dir: str, **kwargs):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    train_dir = os.path.join(data_dir, 'train')
    val_dir = os.path.join(data_dir, 'val')

    train_dataset = datasets.ImageFolder(train_dir, transform=transform)
    val_dataset = datasets.ImageFolder(val_dir, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=kwargs.get('batch_size', 32), shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=kwargs.get('batch_size', 32), shuffle=False)

    # Возможность выбора одной из трех моделей 
    num_classes = len(train_dataset.classes)
    if model_type == 'resnet':
        model = models.resnet18(pretrained=True)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    elif model_type == 'vgg':
        model = models.vgg16(pretrained=True)
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
    elif model_type == 'efficientnet':
        model = models.efficientnet_b0(pretrained=True)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    else:
        raise ValueError("Unsupported model type. Choose from 'resnet', 'vgg', or 'efficientnet'.")

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=kwargs.get('lr', 0.001))

    # Выполняет обучение алгоритма машинного обучения на обучающем наборе данных
    epochs = kwargs.get('epochs', 10)
    start_time = time.time()

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_loader)}")

    training_time = time.time() - start_time

    # Проводит оценку точности на валидационной выборке
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')
    roc_auc = roc_auc_score(all_labels, all_preds, multi_class='ovr')

    metrics = {
        'accuracy': accuracy,
        'roc_auc_score': roc_auc,
        'f1_score_macro': f1
    }

    # 6. Подготовка результатов
    results = {
        'model': model,
        'metrics': metrics,
        'training_time_per_epoch': training_time / epochs,
        'total_training_time_minutes': training_time / 60
    }

    # 7. Сохранение результатов
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    save_path = f"model_results_{timestamp}.pt"
    torch.save(results, save_path)

    return results

In [31]:
data_directory = "temp_dir"  # Укажите путь к вашей директории с данными
hyperparameters = {
    'model_type': 'resnet',
    'batch_size': 8,
    'lr': 0.01,
    'epochs': 1
}

results = train_model(
    data_dir=data_directory,
    **hyperparameters
)

print("Metrics:", results['metrics'])
print("Training time per epoch:", results['training_time_per_epoch'], "seconds")
print("Total training time:", results['total_training_time_minutes'], "minutes")

KeyboardInterrupt: 

In [32]:
from ultralytics import YOLO

In [56]:
# Load the model.
model = YOLO('yolov8n.pt')
 
# Training.
results = model.train(
   data='custom_data.yaml',
   imgsz=224,
   epochs=1,
   batch=8,
   name='yolov8n_custom')

Ultralytics 8.3.70 🚀 Python-3.12.7 torch-2.6.0+cu124 CPU (13th Gen Intel Core(TM) i5-13400)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=custom_data.yaml, epochs=1, time=None, patience=100, batch=8, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n_custom13, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, s

train: Scanning /home/c6/new_main_venv/new_temp_dir/train.cache... 0 images, 23 backgrounds, 0 corrupt: 100%|██████████| 23/23 [00:00<?, ?it/s]

WARNING ⚠️ No labels found in /home/c6/new_main_venv/new_temp_dir/train.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.



val: Scanning /home/c6/new_main_venv/new_temp_dir/val.cache... 0 images, 4 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]

WARNING ⚠️ No labels found in /home/c6/new_main_venv/new_temp_dir/val.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
Plotting labels to runs/detect/yolov8n_custom13/labels.jpg... 
zero-size array to reduction operation maximum which has no identity
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 224 train, 224 val
Using 0 dataloader workers
Logging results to runs/detect/yolov8n_custom13
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all          4          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels



1 epochs completed in 0.001 hours.
Optimizer stripped from runs/detect/yolov8n_custom13/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/yolov8n_custom13/weights/best.pt, 6.2MB

Validating runs/detect/yolov8n_custom13/weights/best.pt...
Ultralytics 8.3.70 🚀 Python-3.12.7 torch-2.6.0+cu124 CPU (13th Gen Intel Core(TM) i5-13400)
Model summary (fused): 168 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 15.56it/s]

                   all          4          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels


Speed: 0.1ms preprocess, 14.8ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/detect/yolov8n_custom13


In [73]:
import os

def get_folder_names_and_save(folder_path, output_file="classes.txt"):
    """
    Функция принимает путь к папке, получает имена папок внутри неё,
    и записывает их в файл в формате 'names: [...]'.

    :param folder_path: Путь к главной папке
    :param output_file: Имя файла для записи результата (по умолчанию 'classes.txt')
    """
    # Проверяем, существует ли путь и является ли он папкой
    if not os.path.exists(folder_path):
        raise FileNotFoundError(f"Путь '{folder_path}' не существует.")
    if not os.path.isdir(folder_path):
        raise NotADirectoryError(f"Путь '{folder_path}' не является папкой.")

    # Получаем список всех элементов в папке
    all_items = os.listdir(folder_path)

    # Фильтруем только папки
    folder_names = [item for item in all_items if os.path.isdir(os.path.join(folder_path, item))]

    # Формируем результат в требуемом формате
    result = {"names": folder_names}

    # Записываем результат в файл
    with open(output_file, "w", encoding="utf-8") as file:
        file.write(f"names: {result['names']}")

    print(f"Результат успешно записан в файл '{output_file}'.")


get_folder_names_and_save('temp_dir/val/')


Результат успешно записан в файл 'classes.txt'.


In [89]:
def train_yolo_model(**kwargs):
    # Извлечение имени модели из kwargs
    model_name = kwargs.pop('model_name')  # Удаляем model_name из kwargs и сохраняем в переменной
    
    # Возможность выбора одной из трех моделей
    if model_name in ["yolov8n.pt", "yolov8m.pt", "yolov8s.pt"]:
        print("Переменная равна одному из значений.")
    
        print("Переменная не равна ни одному из значений.")
        # Загрузка модели
        model = YOLO(model_name)
        
        # Обучение модели
        start_time = time.time()
        results = model.train(
            data='custom_data.yaml',
            imgsz=kwargs.get('imgsz', 640),  # Параметр по умолчанию 640, если не указан
            epochs=kwargs.get('epochs', 50),  # Параметр по умолчанию 50, если не указан
            batch=kwargs.get('batch', 16),    # Параметр по умолчанию 16, если не указан
            name='yolov8n_custom'
        )
        # Засекаем конечное время
        end_time = time.time()

        # Вычисляем разницу во времени (время выполнения функции)
        execution_time = end_time - start_time
        # Сохранение модели
        saved_model_path = model.export(format='onnx')  # Пример сохранения в формате ONNX
        print(f"Модель сохранена по пути: {saved_model_path}")
        
        # Запись имени модели в файл log.txt
        with open('log.txt', 'a') as log_file:  # Открываем файл в режиме добавления ('a')
            log_file.write(f"Model Name: {model_name}, Saved Path: {saved_model_path}\n")
        print(f"Model Name: {model_name}, Saved Path: {saved_model_path}\n")
        
        # запись время выполнения
        with open('log.txt', 'a') as file:
            file.write(f'Время выполнения функции: {execution_time:.4f} секунд\n')
        file.print(f'Время выполнения функции: {execution_time:.4f} секунд\n')
    
        return results
    else:
        return 0

In [91]:
results = train_yolo_model(
    model_name='yolov8n.pt',
    imgsz=640,
    epochs=1,
    batch=16,
)

# print("Точности за каждую эпоху:")
# print(results.eval_metrics)
# print("Метрики за каждую эпоху:")
# for epoch in results.metrics:
#     print(epoch)

Переменная равна одному из значений.
Переменная не равна ни одному из значений.
Ultralytics 8.3.70 🚀 Python-3.12.7 torch-2.6.0+cu124 CPU (13th Gen Intel Core(TM) i5-13400)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=custom_data.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n_custom38, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save

train: Scanning /home/c6/new_main_venv/new_temp_dir/train.cache... 0 images, 23 backgrounds, 0 corrupt: 100%|██████████| 23/23 [00:00<?, ?it/s]

WARNING ⚠️ No labels found in /home/c6/new_main_venv/new_temp_dir/train.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.



val: Scanning /home/c6/new_main_venv/new_temp_dir/val.cache... 0 images, 4 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]

WARNING ⚠️ No labels found in /home/c6/new_main_venv/new_temp_dir/val.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
Plotting labels to runs/detect/yolov8n_custom38/labels.jpg... 
zero-size array to reduction operation maximum which has no identity
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000833, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/yolov8n_custom38
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all          4          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels



1 epochs completed in 0.003 hours.
Optimizer stripped from runs/detect/yolov8n_custom38/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/yolov8n_custom38/weights/best.pt, 6.2MB

Validating runs/detect/yolov8n_custom38/weights/best.pt...
Ultralytics 8.3.70 🚀 Python-3.12.7 torch-2.6.0+cu124 CPU (13th Gen Intel Core(TM) i5-13400)
Model summary (fused): 168 layers, 3,007,208 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.12it/s]

                   all          4          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels


Speed: 1.9ms preprocess, 108.6ms inference, 0.0ms loss, 4.4ms postprocess per image
Results saved to runs/detect/yolov8n_custom38
Ultralytics 8.3.70 🚀 Python-3.12.7 torch-2.6.0+cu124 CPU (13th Gen Intel Core(TM) i5-13400)
Model summary (fused): 168 layers, 3,007,208 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'runs/detect/yolov8n_custom38/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 12, 8400) (6.0 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.48...
ONNX: export success ✅ 0.7s, saved as 'runs/detect/yolov8n_custom38/weights/best.onnx' (11.7 MB)

Export complete (0.9s)
Results saved to /home/c6/new_main_venv/runs/detect/yolov8n_custom38/weights
Predict:         yolo predict task=detect model=runs/detect/yolov8n_custom38/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=runs/detect/yolov8n_custom38/weights/best.onnx imgsz=640 data=custom_data.yaml  
Visualize:       https: